In [2]:
import pickle
import numpy as np
import pandas as pd
import torch 
from torch import nn
# import torch_geometric
from torch_geometric.nn import SAGEConv
# from torch_geometric.data import Data
import networkx as nx
import numpy as np
import pandas as pd 
import pickle
import pdb
from torch_geometric.loader import DataLoader
# import torch.optim as optim
import torch.nn.functional as F
# from torch_geometric.utils import to_dense_adj, subgraph, k_hop_subgraph
import matplotlib.pyplot as plt
from tools.combine_nx_to_dataloader import GraphDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [32]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim=40, encoded_dim=3):
        super(Autoencoder, self).__init__()
        
        # Encoder
        # Adjusted to include two additional layers
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 30),  # Reducing from input dimension to 30
            nn.ReLU(True),
            nn.Linear(30, 25),  # Additional layer reducing to 25 dimensions
            nn.ReLU(True),
            nn.Linear(25, 20),  # Additional layer reducing to 20 dimensions
            nn.ReLU(True),
            nn.Linear(20, encoded_dim)  # Final layer reducing to the encoded dimensions
        )
        
        # Decoder
        # Adjusted to include two additional layers
        self.decoder = nn.Sequential(
            nn.Linear(encoded_dim, 20),  # Expanding from encoded dimensions to 20
            nn.ReLU(True),
            nn.Linear(20, 25),  # Additional layer expanding to 25 dimensions
            nn.ReLU(True),
            nn.Linear(25, 30),  # Additional layer expanding to 30 dimensions
            nn.ReLU(True),
            nn.Linear(30, input_dim),  # Final layer to reconstruct the original input size
            nn.Sigmoid()  # Using sigmoid to ensure the output is between 0 and 1
        )
    
    
    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return recon, z


In [7]:
import os
import pickle

In [18]:

# Example usage
node_list = []
folder_graph = './graphs/graphsage_graph/single_graphs/'
metadata_file = folder_graph + 'metadata.csv'
succTimeFiles = folder_graph + 'graphMetadata.csv'

files = os.listdir(folder_graph)
files = [file for file in files if file.endswith('.pickle')]
for file in files:
    fil =  open(folder_graph+file, 'rb')
    G = pickle.load(fil)
    fil.close()
    # print(G.nodes[0])
    for node in G.nodes(data=True):
        # print(node[1]['x'])
        # break
        node_list.append(node[1]['x'])
    # break

# dataset = GraphDataset(graph_list)
# dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Save the graph list for later use, similar to the previous example
torch.save(node_list, 'nodelist.pth')

# # Example of iterating over the DataLoader in a training loop
# for node_features, edge_index, edge_features in dataloader:
#     # Use these in your GAE model
#     pass

In [19]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
folder_graph = './graphs/graphsage_graph/single_graphs/'
fname = 'nodelist.pth'
nodelist = torch.load(fname)
data_nodes = torch.tensor(nodelist, dtype=torch.float32)
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Return the item at the given index
        return self.data[index]
dataset = CustomDataset(data_nodes)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [33]:
import gc
# Initialize the autoencoder
generated_embeds = []
losses = []
for ed in range(3,10):
    autoencoder = Autoencoder(encoded_dim=ed)

    # Example of a loss function and optimizer
    criterion = nn.MSELoss() # Mean Squared Error Loss
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.1) # Adam optimizer
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5, verbose=True)

    # d = [a for a in dataloader]
    # print(d)
    loss_vec = []
    embed_vec = []
    for epoch in range(1, 50):
        total_loss = 0.0
        for input_vector in dataloader:  # Assuming the dataset returns a tuple where the first element is the input
            optimizer.zero_grad()
            output, embed = autoencoder(input_vector)
            # print(input_vector[0], '\n', output)
            loss = criterion(output, input_vector)  # Assuming target is the same as input
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Average loss for this epoch
        epoch_loss = total_loss / len(dataloader)
        embed_vec.append(embed)
        loss_vec.append(epoch_loss)
        # Step the scheduler based on the epoch loss
        scheduler.step(epoch_loss)
        gc.collect()
        # if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {epoch_loss:.4f}')
    losses.append(loss_vec)
    generated_embeds.append(embed_vec)

Epoch 1, Loss: 1.6416
Epoch 2, Loss: 1.6414
Epoch 3, Loss: 1.6414
Epoch 4, Loss: 1.6415
Epoch 5, Loss: 1.6414
